In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

2024-11-06 04:16:34.399785: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 04:16:35.263758: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataprep import *
#from dataloaders.OptimizedDataGenerator import OptimizedDataGenerator
from loss import *
from models.models_bnorm import *

# Scaling Lists for Different Pixel Pitches:
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.22]
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.22]
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.22]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.22]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.22]
* 50x25x100 um:   [75.0, 15.0, 10.0, 1.22]

In [3]:
batch_size = 100
val_batch_size = 100
train_file_size = 40
val_file_size = 10

training_generator = CustomDataGenerator(
    data_directory_path = "/data/dajiang/smartPixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/recon3D/",
    labels_directory_path = "/data/dajiang/smartPixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
    shuffle= True

)

validation_generator = CustomDataGenerator(
    data_directory_path = "/data/dajiang/smartPixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/recon3D/",
    labels_directory_path = "/data/dajiang/smartPixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
    shuffle= True
)

Processing Files: 100%|██████████| 10/10 [01:44<00:00, 10.42s/it]


In [4]:
# compiles model
model=CreateModel((13,21,20),n_filters=5,pool_size=3)
model.summary()

2024-11-06 04:32:31.174898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3234 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:81:00.0, compute capability: 8.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         285       
 ableConv2D)                                                     
                                                                 
 q_batch_normalization (QBa  (None, 11, 19, 5)         20        
 tchNormalization)                                               
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_batch_normalization_1 (Q  (None, 11, 19, 5)         20    

In [5]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [ ]:
# training
base_dir = "./weights-50x12P5-bs100-custom-checkpoints"
os.mkdir(base_dir)
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

class ScalePrintingCallback(keras.callbacks.Callback):    
    def on_epoch_end(self, epoch, logs=None):
        scale_layer = self.model.layers[-1]
        print(
            f"scaling layer ({epoch}):", 
            scale_layer.scale, 
            tf.math.softplus(scale_layer.scale)
        )

print_scale = ScalePrintingCallback()

history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=600,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=2)

Epoch 1/600


2024-11-06 04:33:05.265410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2024-11-06 04:33:05.329176: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-06 04:33:05.480010: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-11-06 04:33:05.490630: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x88a7e90
2024-11-06 04:33:05.506418: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f667f0cd700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-06 04:33:05.506450: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.5gb, Compute Capability 8.0
2024-11-06 04:33:05.511043: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir

19902/19902 - 862s - loss: -5.3609e+01 - val_loss: -1.7453e+02 - 862s/epoch - 43ms/step
Epoch 2/600
